In [12]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Load the datasets
ilk_is_bulma_suresi_df = pd.read_csv('ilk_iş_bulma_süresi_cleraed.csv')
istihdam_orani_df = pd.read_csv('istihdam_oranı_cleared.csv')
kazanc_grubu_df = pd.read_csv('kazanç_grubu_cleared.csv')
yok_with_fees_df = pd.read_csv('test2.csv')

# Function to get the best match for a faculty or department name
def get_best_match(name, choices, threshold=70):
    result = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
    if result:
        match, score = result  
        return match if score >= threshold else None
    return None


# Function to merge datasets with fuzzy matching and nearest academic year
def merge_datasets(df1, df2, dept_or_faculty_matches):
    for dept_or_faculty in dept_or_faculty_matches:
        df1['matched_name'] = df1['Program'].apply(lambda x: get_best_match(x, df2[dept_or_faculty].unique()))
        for index, row in df2.iterrows():
            matched_rows = df1.loc[df1['matched_name'] == row[dept_or_faculty]]
            if not matched_rows.empty:
                if row['academicYear'] <= 2022:
                    df2.at[index, 'avg_monthly_income_group'] = matched_rows['2022'].values[0]
                elif row['academicYear'] >= 2023:
                    df2.at[index, 'avg_monthly_income_group'] = matched_rows['2023'].values[0]
        df1.drop(columns=['matched_name'], inplace=True)
    return df2
# Attempt matches with both department and faculty names
dept_or_faculty_matches = ['departmentName', 'faculty']

# Merge the datasets
merged_kazanc_grubu_df = merge_datasets(kazanc_grubu_df, yok_with_fees_df, dept_or_faculty_matches)

# Save the updated DataFrames to new CSV files
merged_kazanc_grubu_df.to_csv("final_tuik.csv", index=False)

/var/folders/lz/72v2qwyx34g0fk80lsqg3qt80000gn/T/ipykernel_16907/1081871938.py:8: DtypeWarning: Columns (49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  yok_with_fees_df = pd.read_csv('test2.csv')
